In [ ]:
#! pip install joblib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import os
import cv2
import matplotlib.patches as patches
from lmfit import Model, Parameters
from mrs_pack import MRSpack
from mrs_pack import functions
from tqdm import tqdm
from joblib import Parallel, delayed
from astropy.cosmology import FlatLambdaCDM
from scipy.constants import c
from scipy.interpolate import interp1d

# COSMOLOGY USED
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
###############

In [ ]:
# Gaussian function with peak, sigma and mean
def mygauss(x,mean, sig, peak):
    exp_in = (x - mean)**2/(2*sig**2)
    return peak*np.exp(-exp_in)

# Extract spectrum from a circular region from a fits cube given the center in X and Y and radius in R (pixels)
def spec_circ_aper(cube, x0, y0, R):
    Nz, Ny , Nx = np.shape(cube)
    x,y = np.arange(Nx), np.arange(Ny)
    xgrid, ygrid = np.meshgrid(x,y)
    rgrid = ((xgrid-x0)**2+(ygrid-y0)**2)**0.5
    w = np.where(rgrid < R)
    spec = np.zeros(Nz)
    for k in range(Nz):
        img = cube[k,:,:]
        spec[k] = img[w].sum()
    return spec

# Given flux, t gives luminosity
def luminosity(flux, redshift):
    d_l = cosmo.luminosity_distance(redshift)
    area = 4*np.pi*d_l.value*d_l.value
    return np.log10(flux)+np.log10(area)+2.*(24.+np.log10(3.08567802))

In [ ]:
z = 0.1 # Redshift of the target
muse_raw = fits.open("muse_raw.fits") # Raw MUSE cube
muse_models = fits.open("muse_cont_model.fits") # Continuum model

In [ ]:
# Visualise how the spectrum looks like for one of the pixels

i_test, j_test = 169, 263
plt.plot(muse_raw[0].data[:,j_test,i_test])
plt.plot(muse_models[1].data[:,j_test,i_test])

In [ ]:
# Make the continuum-subtracted cube to get an emission line only cube
contsub_cube = muse_raw[0].data-muse_models[1].data
Nz_muse,Ny_muse,Nx_muse = np.shape(contsub_cube)
muse_pscale = abs(muse_raw[0].header["CD1_1"])*3600 # MUSE pixel scale from the header

In [ ]:
# Visualise the continuum-subtracted spectrum from the spectrum plotted above

plt.plot(contsub_cube[:,j_test,i_test])

In [ ]:
# Get the wavelength array from MUSE headers

wl = (np.linspace(1,Nz_muse,Nz_muse)-muse_raw[0].header["CRPIX3"])*muse_raw[0].header["CDELT3"]+\
    muse_raw[0].header["CRVAL3"]
wl_DR = wl/(1+z) # Wavelength array in de-redshifted space

In [ ]:
# Verify the wavelength array using the spectrum above. The [OIII]5007 location should match

plt.plot(wl_DR, contsub_cube[:,j_test,i_test])
plt.axvline(5006.843,color="black",alpha=0.4)

In [ ]:
# Make the median MUSE image (Collapsing along the spectral axis)

fig, ax = plt.subplots(figsize=(7,6))

ax.imshow(np.sum(muse_raw[0].data,axis=0),origin="lower",cmap='bone',
          vmin=0,vmax=8e+5,
         extent=[-muse_pscale*Nx_muse/2,muse_pscale*Nx_muse/2,-muse_pscale*Ny_muse/2,+muse_pscale*Ny_muse/2])

# Add a circular aperture patch (optional)
#aper_print = patches.Circle((0.35,-0.6),1.5, linewidth=4, edgecolor='red',facecolor="none")
#ax.add_patch(aper_print)

ax.set_xlabel("$\Delta$x [arcsec]", fontsize=16)
ax.set_ylabel("$\Delta$y [arcsec]", fontsize=16)
ax.tick_params(labelsize=14)
ax.set_xlim(-10,10)
ax.set_ylim(-10,10)
plt.tight_layout()
plt.savefig("/Users/dk24abv/work/papers/GOALS/GOALS_kakkad/plots/MUSE_median_img.png")

In [ ]:
# Get the emission line locations
line_lists = json.load(open('/Users/dk24abv/work/software/line_list.json'))
opt_lines = line_lists["optical"]
print(opt_lines)

# Access the first element
element_dict = opt_lines[0]  # The first element in the JSON list

# Set variables dynamically from the dictionary
for element, value in element_dict.items():
    globals()[element] = value  # Sets the variable name as the element key

In [ ]:
# Different functions used for modelling emission lines

def fit_func_1G(x,c1,c2,mean,sig,peak):
    cont_comp = c1 + c2*x
    line_comp = mygauss(x,mean,sig,peak)
    return cont_comp + line_comp

def fit_func_2G(x,c1,c2,mean1,sig1,peak1,mean2,sig2,peak2):
    cont_comp = c1 + c2*x
    line_comp1 = mygauss(x,mean1,sig1,peak1)
    line_comp2 = mygauss(x,mean2,sig2,peak2)
    return cont_comp + line_comp1 + line_comp2

def func_oiii_hb(x,c1, c2, mean, sig, peak_oiii, peak_hb):
    comp_cont = c1 + c2*x
    comp_oiii5007 = mygauss(x,mean,sig,peak_oiii)
    comp_oiii4959 = mygauss(x,mean+(OIII4959-OIII5007),sig,peak_oiii/3)
    comp_hbeta = mygauss(x,mean+(Hbeta-OIII5007),sig,peak_hb)
    full_model = comp_cont+comp_oiii5007+comp_oiii4959+comp_hbeta
    return full_model

def func_oiii_hb_2G(x,c1,c2,mean1,sig1,peak1_oiii,peak1_hb,mean2,sig2,peak2_oiii,peak2_hb):
    comp_cont = c1 + c2*x
    comp_oiii5007 = mygauss(x,mean1,sig1,peak1_oiii)
    comp_oiii4959 = mygauss(x,mean1+(OIII4959-OIII5007),sig1,peak1_oiii/3)
    comp_hbeta = mygauss(x,mean1+(Hbeta-OIII5007),sig1,peak1_hb)
    comp_oiii5007b = mygauss(x,mean2,sig2,peak2_oiii)
    comp_oiii4959b = mygauss(x,mean2+(OIII4959-OIII5007),sig2,peak2_oiii/3)
    comp_hbetab = mygauss(x,mean2+(Hbeta-OIII5007),sig2,peak2_hb)
    narrow_model = comp_oiii5007+comp_oiii4959+comp_hbeta
    broad_model = comp_oiii5007b+comp_oiii4959b+comp_hbetab
    full_model = comp_cont+narrow_model+broad_model
    return full_model

def func_nii_ha(x,c1,c2,mean,sig,peak_ha,peak_nii):
    comp_cont = c1 + c2*x
    comp_halpha = mygauss(x,mean,sig,peak_ha)
    comp_nii6583 = mygauss(x,mean+(NII6583-Halpha),sig,peak_nii)
    comp_nii6548 = mygauss(x,mean+(NII6548-Halpha),sig,peak_nii/3)
    full_model = comp_cont+comp_halpha+comp_nii6583+comp_nii6548
    return full_model

def func_nii_ha_2G(x,c1,c2,mean1,sig1,peak1_ha,peak1_nii,mean2,sig2,peak2_ha,peak2_nii):
    comp_cont = c1 + c2*x
    comp_halpha = mygauss(x,mean1,sig1,peak1_ha)
    comp_nii6583 = mygauss(x,mean1+(NII6583-Halpha),sig1,peak1_nii)
    comp_nii6548 = mygauss(x,mean1+(NII6548-Halpha),sig1,peak1_nii/3)
    comp_halphab = mygauss(x,mean2,sig2,peak2_ha)
    comp_nii6583b = mygauss(x,mean2+(NII6583-Halpha),sig2,peak2_nii)
    comp_nii6548b = mygauss(x,mean2+(NII6548-Halpha),sig2,peak2_nii/3)
    full_model = comp_cont+comp_halpha+comp_nii6583+comp_nii6548+comp_halphab+comp_nii6583b+comp_nii6548b
    return full_model

def func_sii(x,c1,c2,mean,sig,peak_sii6716,peak_sii6731):
    comp_cont = c1 + c2*x
    comp_sii6716 = mygauss(x,mean,sig,peak_sii6716)
    comp_sii6731 = mygauss(x,mean+(SII6731-SII6716),sig,peak_sii6731)
    full_model = comp_cont+comp_sii6716+comp_sii6731
    return full_model

def func_sii_2G(x,c1,c2,mean1,sig1,peak1_sii6716,peak1_sii6731,mean2,sig2,peak2_sii6716,peak2_sii6731):
    comp_cont = c1 + c2*x
    comp_sii6716 = mygauss(x,mean1,sig1,peak1_sii6716)
    comp_sii6731 = mygauss(x,mean1+(SII6731-SII6716),sig1,peak1_sii6731)
    comp_sii6716b = mygauss(x,mean2,sig2,peak2_sii6716)
    comp_sii6731b = mygauss(x,mean2+(SII6731-SII6716),sig2,peak2_sii6731)
    full_model = comp_cont+comp_sii6716+comp_sii6731+comp_sii6716b+comp_sii6731b
    return full_model

def compute_v_percentiles(wavelength, flux, line_center_rest):
    """
    Compute v10, v50, v90 for an emission line in a spectrum.

    Parameters:
    - wavelength: array of wavelengths [in Angstroms]
    - flux: array of fluxes (same length as wavelength)
    - line_center_rest: rest-frame wavelength of the emission line [in Angstroms]

    Returns:
    - v10, v50, v90: velocities in km/s at 10%, 50%, 90% cumulative line flux
    """
    # Ensure arrays are sorted by wavelength
    sorted_indices = np.argsort(wavelength)
    wavelength = wavelength[sorted_indices]
    flux = flux[sorted_indices]

    # Subtract continuum if needed (optional: here a simple median is used)
    continuum = np.median(flux)
    line_flux = flux - continuum
    line_flux[line_flux < 0] = 0  # set negative values to zero

    # Compute cumulative flux
    cum_flux = np.cumsum(line_flux)
    cum_flux /= cum_flux[-1]  # normalize to 1

    # Interpolate wavelength at which cumulative flux reaches 0.1, 0.5, 0.9
    f_interp = interp1d(cum_flux, wavelength, bounds_error=False, fill_value="extrapolate")
    wl10 = f_interp(0.1)
    wl50 = f_interp(0.5)
    wl90 = f_interp(0.9)

    # Convert wavelengths to velocity offsets from the line center
    def wl_to_v(wl): return ((wl - line_center_rest) / line_center_rest) * c / 1000  # km/s

    v10 = wl_to_v(wl10)
    v50 = wl_to_v(wl50)
    v90 = wl_to_v(wl90)

    return v10, v50, v90

In [ ]:
# Fit a one Gaussian model to OIII & Hbeta
contsub_cube[np.isnan(contsub_cube)] = 0
filt_oiii = np.where((wl_DR>4800)&(wl_DR<5050))
wl_oiii = wl_DR[filt]
cube_oiii = contsub_cube[filt,:,:][0,:,:,:]

oiii_wave_1G = np.zeros((Ny_muse,Nx_muse))
oiii_sig_1G = np.zeros((Ny_muse,Nx_muse))
oiii_flux_1G = np.zeros((Ny_muse,Nx_muse))
Hb_flux_1G = np.zeros((Ny_muse,Nx_muse))
bic_oiii_1G = np.zeros((Ny_muse, Nx_muse))

model_oiii = Model(func_oiii_hb)
params_oiii = Parameters()
params_oiii.add('c1', value=0)
params_oiii.add('c2', value=0)
params_oiii.add('mean', value=5006.843, min=4990, max=5020)
params_oiii.add('sig', value=1.5, min=0)
params_oiii.add('peak_oiii', value=1, min=0)
params_oiii.add('peak_hb', value=1, min=0)

def fit_pixel_oiii_1G(j, i):
    # You can remove the following line if you wish. This is a running mean of neighbouring 5 pixels to increase the overall SNR of the cube
    # This does not compromise with the PSF or resolution of the observations
    spec = (cube_oiii[:,j,i]+cube_oiii[:,j-1,i]+cube_oiii[:,j+1,i]+cube_oiii[:,j,i-1]+cube_oiii[:,j,i+1])/5 
    spec_norm = spec / np.max(spec) if np.max(spec) != 0 else spec
    spec_norm = np.nan_to_num(spec_norm, nan=0.0)
    fit_oiii = model_oiii.fit(spec_norm, params_oiii, x=wl_oiii, weights=1)
    return (
        fit_oiii.params["mean"].value,
        fit_oiii.params["sig"].value,
        np.sqrt(2*np.pi)*fit_oiii.params["sig"].value*fit_oiii.params["peak_oiii"].value*1e-20*np.max(spec),
        np.sqrt(2*np.pi)*fit_oiii.params["sig"].value*fit_oiii.params["peak_hb"].value*1e-20*np.max(spec),
        fit_oiii.bic
    )

# Create the list of pixel coordinates to process
coords = [(j, i) for j in range(1, Ny_muse-1) for i in range(1, Nx_muse-1)]

# Parallel with tqdm
results = Parallel(n_jobs=-1, prefer="threads")(
    delayed(fit_pixel_oiii_1G)(j, i)
    for j, i in tqdm(coords, desc="Fitting pixels")
)

# Fill the output arrays
idx = 0
for j, i in coords:
    oiii_wave_1G[j,i], oiii_sig_1G[j,i], oiii_flux_1G[j,i], hb_flux_1G[j,i], bic_oiii_1G[j,i] = results[idx]
    idx += 1

In [ ]:
# Fit a two Gaussian model to OIII & Hbeta
oiii_wave1_2G = np.zeros((Ny_muse,Nx_muse))
oiii_sig1_2G = np.zeros((Ny_muse,Nx_muse))
oiii_flux1_2G = np.zeros((Ny_muse,Nx_muse))
Hb_flux1_2G = np.zeros((Ny_muse,Nx_muse))
oiii_wave2_2G = np.zeros((Ny_muse,Nx_muse))
oiii_sig2_2G = np.zeros((Ny_muse,Nx_muse))
oiii_flux2_2G = np.zeros((Ny_muse,Nx_muse))
Hb_flux2_2G = np.zeros((Ny_muse,Nx_muse))
bic_oiii_2G = np.zeros((Ny_muse, Nx_muse))

model_oiii2G = Model(func_oiii_hb_2G)
params_oiii2G = Parameters()
params_oiii2G.add('c1', value=0)
params_oiii2G.add('c2', value=0)
params_oiii2G.add('mean1', value=5006.843, min=4990, max=5020)
params_oiii2G.add('sig1', value=1.5, min=0)
params_oiii2G.add('peak1_oiii', value=1, min=0)
params_oiii2G.add('peak1_hb', value=1, min=0)
params_oiii2G.add('mean2', value=5006.843, min=4990, max=5020)
params_oiii2G.add('sig2', value=3, min=0)
params_oiii2G.add('peak2_oiii', value=0.5, min=0)
params_oiii2G.add('peak2_hb', value=0.5, min=0)

def fit_pixel_oiii_2G(j, i):
    # You can remove the following line if you wish. This is a running mean of neighbouring 5 pixels to increase the overall SNR of the cube
    # This does not compromise with the PSF or resolution of the observations
    spec = (cube_oiii[:,j,i]+cube_oiii[:,j-1,i]+cube_oiii[:,j+1,i]+cube_oiii[:,j,i-1]+cube_oiii[:,j,i+1])/5 
    spec_norm = spec / np.max(spec) if np.max(spec) != 0 else spec
    spec_norm = np.nan_to_num(spec_norm, nan=0.0)
    fit_oiii = model_oiii2G.fit(spec_norm, params_oiii, x=wl_oiii, weights=1)
    return (
        fit_oiii.params["mean1"].value,
        fit_oiii.params["sig1"].value,
        np.sqrt(2*np.pi)*fit_oiii.params["sig1"].value*fit_oiii.params["peak1_oiii"].value*1e-20*np.max(spec),
        np.sqrt(2*np.pi)*fit_oiii.params["sig1"].value*fit_oiii.params["peak1_hb"].value*1e-20*np.max(spec),
        fit_oiii.params["mean2"].value,
        fit_oiii.params["sig2"].value,
        np.sqrt(2*np.pi)*fit_oiii.params["sig2"].value*fit_oiii.params["peak2oiii"].value*1e-20*np.max(spec),
        np.sqrt(2*np.pi)*fit_oiii.params["sig2"].value*fit_oiii.params["peak2_hb"].value*1e-20*np.max(spec),
        fit_oiii.bic
    )

# Parallel with tqdm
results = Parallel(n_jobs=-1, prefer="threads")(
    delayed(fit_pixel_oiii_2G)(j, i)
    for j, i in tqdm(coords, desc="Fitting pixels")
)

# Fill the output arrays
idx = 0
for j, i in coords:
    oiii_wave1_2G[j,i], oiii_sig1_2G[j,i], oiii_flux1_2G[j,i], hb_flux1_2G[j,i], oiii_wave2_2G[j,i], oiii_sig2_2G[j,i], oiii_flux2_2G[j,i], hb_flux2_2G[j,i], bic_oiii_1G[j,i] = results[idx]
    idx += 1

In [ ]:
# Create a Primary HDU object
primary_data = np.ones((1, 1))
primary_hdu = fits.PrimaryHDU(data=primary_data)

# Create Image HDUs for the extensions
oiii_wave_1G_hdu = fits.ImageHDU(data=oiii_wave_1G,name='OIII wave 1G')
oiii_sig_1G_hdu = fits.ImageHDU(data=oiii_sig_1G,name='OIII sigma 1G')
oiii_flux_1G_hdu = fits.ImageHDU(data=oiii_flux_1G,name='OIII flux 1G')
Hbeta_flux_1G_hdu = fits.ImageHDU(data=hb_flux_1G,name='Hbeta flux 1G')
bic_oiii_1G_hdu = fits.ImageHDU(data=bic_oiii_1G,name='BIC oiii 1G')

oiii_wave1_2G_hdu = fits.ImageHDU(data=oiii_wave1_2G,name='OIII wave1 2G')
oiii_sig1_2G_hdu = fits.ImageHDU(data=oiii_sig1_2G,name='OIII sigma1 2G')
oiii_flux1_2G_hdu = fits.ImageHDU(data=oiii_flux1_2G,name='OIII flux1 2G')
Hbeta_flux1_2G_hdu = fits.ImageHDU(data=hb_flux1_2G,name='Hbeta flux1 2G')
oiii_wave2_2G_hdu = fits.ImageHDU(data=oiii_wave2_2G,name='OIII wave2 2G')
oiii_sig2_2G_hdu = fits.ImageHDU(data=oiii_sig2_2G,name='OIII sigma2 2G')
oiii_flux2_2G_hdu = fits.ImageHDU(data=oiii_flux2_2G,name='OIII flux2 2G')
Hbeta_flux2_2G_hdu = fits.ImageHDU(data=hb_flux2_2G,name='Hbeta flux2 2G')
bic_oiii_2G_hdu = fits.ImageHDU(data=bic_oiii_2G,name='BIC oiii 2G')


# Create an HDUList to contain all HDUs
hdulist = fits.HDUList([primary_hdu, oiii_wave_1G_hdu, oiii_sig_1G_hdu, oiii_flux_1G_hdu, Hbeta_flux_1G_hdu, bic_oiii_1G_hdu,
                        oiii_wave1_2G_hdu, oiii_sig1_2G_hdu, oiii_flux1_2G_hdu, Hbeta_flux1_2G_hdu, oiii_wave2_2G_hdu, 
                        oiii_sig2_2G_hdu, oiii_flux2_2G_hdu, Hbeta_flux2_2G_hdu, bic_oiii_2G_hdu])

# Write the HDUList to a new FITS file
hdulist.writeto('MUSE_maps_DATE.fits', overwrite=True)

In [ ]:
# Add H-alpha, NII and SII models here based on the OIII and H-beta templates above

In [ ]:
# Fit Halpha with one Gaussian

filt_halpha = np.where((wl_DR>6562-50)&(wl_DR<6562+50))
wl_halpha = wl_DR[filt_halpha]
cube_halpha = contsub_cube[filt_halpha,:,:][0,:,:,:]

halpha_wave = np.zeros((Ny_muse,Nx_muse))
halpha_sig = np.zeros((Ny_muse,Nx_muse))
halpha_flux = np.zeros((Ny_muse,Nx_muse))
nii6585_flux = np.zeros((Ny_muse,Nx_muse))

# Model the Halpha line

model_halpha = Model(fit_func_halpha)
params_halpha = Parameters()
params_halpha.add('c1', value=0)
params_halpha.add('c2', value=0)
params_halpha.add('mean', value=6562.8, min=6550, max=6575)
params_halpha.add('sig', value=1.5, min=0)
params_halpha.add('peakHA', value=1, min=0)
params_halpha.add('peakNII', value=1, min=0)

def fit_pixel_halpha(j, i):
    spec = (cube_halpha[:,j,i]+cube_halpha[:,j-1,i]+cube_halpha[:,j+1,i]+cube_halpha[:,j,i-1]+cube_halpha[:,j,i+1])/5
    spec_norm = spec / np.max(spec) if np.max(spec) != 0 else spec
    spec_norm = np.nan_to_num(spec_norm, nan=0.0)
    fit_halpha = model_halpha.fit(spec_norm, params_halpha, x=wl_halpha, weights=1)
    return (
        fit_halpha.params["mean"].value,
        fit_halpha.params["sig"].value,
        np.sqrt(2*np.pi)*fit_halpha.params["sig"].value*fit_halpha.params["peakHA"].value*1e-20*np.max(spec),
        np.sqrt(2*np.pi)*fit_halpha.params["sig"].value*fit_halpha.params["peakNII"].value*1e-20*np.max(spec)
    )

# Parallel with tqdm
results_halpha = Parallel(n_jobs=-1, prefer="threads")(
    delayed(fit_pixel_halpha)(j, i)
    for j, i in tqdm(coords, desc="Fitting Halpha spectra")
)

# Fill the output arrays
idx = 0
for j, i in coords:
    halpha_wave[j, i], halpha_sig[j, i], halpha_flux[j, i], nii6585_flux[j,i] = results_halpha[idx]
    idx += 1